In [1]:
import os
import torch
import datetime
from torchvision.models import resnet18
from torchvision.transforms import v2
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import accuracy_score
from utils import extract_data
import warnings
warnings.filterwarnings('ignore')



In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

In [3]:
train_path = "../12 dataset/train/"
test_path = "../12 dataset/train/"
validation_path = "../12 dataset/valid/"

# Подготовка данных

In [4]:
class CardDataset(Dataset):
    def __init__(self, image_names: str, y, path, transforms=None, normalize=True):
        self.x, self.y = image_names, y
        self.path = path
        self.normalize = normalize
        self.transforms = transforms
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        image_name = self.x[index]
        image = read_image(self.path + image_name).to(dtype=torch.float, device=device)
        label = self.y[index]
        if self.transforms:
            image, label = self.transforms(image, label)
        if self.normalize:
            image = image / 255
        
        return image, label

In [5]:
def create_dataloaders(train_path, validation_path, test_path, transforms=None, mode='all'):
    train_dataset = CardDataset(*extract_data(train_path, mode=mode), path=train_path, transforms=transforms)
    
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    valid_dataset = CardDataset(*extract_data(validation_path, mode=mode), path=validation_path)
    
    valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=True)
    test_dataset = CardDataset(*extract_data(test_path, mode=mode), path=test_path)
    
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)    
    return train_dataloader, valid_dataloader, test_dataloader

In [6]:
transforms = v2.Compose([
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.RandomRotation(45),
    v2.RandomChannelPermutation()
    
])

# Создание модели

In [7]:
def training_loop(n_epochs, optimizer, scheduler, model, loss_fn, train_loader, validation_loader, ):
    best_score = 0
    best_epoch = 0
    counter = 0
    
    for epoch in range(n_epochs):
        model.train()
        for data, label in train_loader:
            data = data.to(device=device)
            label = label.to(device=device)
            output = model(data)
            loss = loss_fn(output, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        scheduler.step()
        model.eval()
        summary_score= 0
        for i, (data, label) in enumerate(validation_loader):
                  
            output = model(data)
            
            summary_score += accuracy_score(label.argmax(1).cpu(), output.argmax(1).cpu())

        validation_score = summary_score/i


        if best_score > validation_score:
            counter += 1
            if counter == 5:
                print(f"Early stop on epoch {epoch}")
                print(f"Weights are loaded from epoch {best_epoch}")
                model.load_state_dict(best_weights)
                break
        else:
            counter = 0
            best_epoch = epoch
            best_score = validation_score
            best_weights = model.state_dict()

        if epoch == 1 or epoch % 5 == 0:

            print('{} Epoch {}, Training loss {}, Validation accuracy {}, lr {}'.format(
                datetime.datetime.now(),
                epoch,
                loss / len(train_loader),
                validation_score,
                scheduler.get_last_lr())
            )

In [8]:
class CustomModel(nn.Module):
    def __init__(self, in_channels, out_channels, image_size):
        super().__init__()
        self.image_size = image_size
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(image_size[0] * image_size[1] // 16 * 8, 32)
        self.fc2 = nn.Linear(32, out_channels)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        #print(out.shape)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        #print(out.shape)
        out = out.view(-1, (self.image_size[0] // 4) * (self.image_size[1] // 4) * 8)
        #print(out.shape)
        out = torch.tanh(self.fc1(out))
        #print(out.shape)
        out = F.softmax(self.fc2(out))
        return out

In [9]:
train_dataloader, valid_dataloader, test_dataloader = create_dataloaders(train_path, validation_path, test_path, transforms=None, mode='all')

In [10]:
resnet_model = resnet18(pretrained=True)
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, 53)
torch.nn.init.xavier_uniform_(resnet_model.fc.weight) 
resnet_model = resnet_model.to(device=device)
optimizer = torch.optim.SGD(resnet_model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10)
loss_fn = nn.CrossEntropyLoss()
training_loop(n_epochs=200, optimizer=optimizer, model=resnet_model, loss_fn=loss_fn, scheduler=scheduler,
              train_loader=train_dataloader, validation_loader=valid_dataloader)
torch.save(resnet_model.state_dict(), 'resnet.pt')

2024-06-02 13:03:12.726141 Epoch 0, Training loss 0.00956401415169239, Validation accuracy 0.03409090909090909, lr [0.0001]
2024-06-02 13:04:49.618564 Epoch 1, Training loss 0.007659234572201967, Validation accuracy 0.06439393939393939, lr [0.0001]
2024-06-02 13:11:34.936948 Epoch 5, Training loss 0.005884621758013964, Validation accuracy 0.23106060606060605, lr [0.0001]
2024-06-02 13:20:03.957026 Epoch 10, Training loss 0.006288644392043352, Validation accuracy 0.3560606060606061, lr [1e-05]
2024-06-02 13:28:39.618925 Epoch 15, Training loss 0.005566319916397333, Validation accuracy 0.38636363636363635, lr [1e-05]
2024-06-02 13:37:08.817674 Epoch 20, Training loss 0.007394666317850351, Validation accuracy 0.4015151515151515, lr [1.0000000000000002e-06]
2024-06-02 13:45:35.758195 Epoch 25, Training loss 0.006321731023490429, Validation accuracy 0.3712121212121212, lr [1.0000000000000002e-06]
2024-06-02 13:54:01.586106 Epoch 30, Training loss 0.006115943193435669, Validation accuracy 0.

# Модель, определяющая масть карты

In [11]:
train_dataloader, valid_dataloader, test_dataloader = create_dataloaders(train_path, validation_path, test_path, mode='suits', transforms=None)

In [12]:
suit_resnet = resnet18(pretrained=True)
suit_resnet.fc = torch.nn.Linear(suit_resnet.fc.in_features, 5)
torch.nn.init.xavier_uniform_(suit_resnet.fc.weight) 
suit_resnet = suit_resnet.to(device=device)
optimizer = torch.optim.SGD(suit_resnet.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10)
loss_fn = nn.CrossEntropyLoss()
training_loop(n_epochs=200, optimizer=optimizer, model=suit_resnet, loss_fn=loss_fn, scheduler=scheduler,
              train_loader=train_dataloader, validation_loader=valid_dataloader)
torch.save(suit_resnet.state_dict(), 'suits_resnet.pt')

2024-06-02 13:57:25.033806 Epoch 0, Training loss 0.0016486214008182287, Validation accuracy 0.6553030303030303, lr [0.0001]
2024-06-02 13:59:07.561119 Epoch 1, Training loss 0.0014357516774907708, Validation accuracy 0.7954545454545454, lr [0.0001]
2024-06-02 14:05:55.386822 Epoch 5, Training loss 0.0008385844412259758, Validation accuracy 0.8901515151515151, lr [0.0001]
2024-06-02 14:14:23.199041 Epoch 10, Training loss 0.0004816995933651924, Validation accuracy 0.9128787878787878, lr [1e-05]
2024-06-02 14:24:06.807578 Epoch 15, Training loss 0.0013270593481138349, Validation accuracy 0.9166666666666666, lr [1e-05]
Early stop on epoch 16
Weights are loaded from epoch 11


# Модель, определяющая значение на карте

In [13]:
train_dataloader, valid_dataloader, test_dataloader = create_dataloaders(train_path, validation_path, test_path, mode='values')

In [14]:
value_resnet = resnet18(pretrained=True)
value_resnet.fc = torch.nn.Linear(value_resnet.fc.in_features, 14)
torch.nn.init.xavier_uniform_(value_resnet.fc.weight) 
value_resnet = value_resnet.to(device=device)
optimizer = torch.optim.SGD(value_resnet.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=10)
loss_fn = nn.CrossEntropyLoss()
training_loop(n_epochs=200, optimizer=optimizer, model=value_resnet, loss_fn=loss_fn, scheduler=scheduler,
              train_loader=train_dataloader, validation_loader=valid_dataloader)
torch.save(value_resnet.state_dict(), 'values_resnet.pt')

2024-06-02 14:28:05.925136 Epoch 0, Training loss 0.004823051393032074, Validation accuracy 0.20454545454545456, lr [0.0001]
2024-06-02 14:30:03.095530 Epoch 1, Training loss 0.004786588251590729, Validation accuracy 0.29924242424242425, lr [0.0001]
2024-06-02 14:37:57.248611 Epoch 5, Training loss 0.004153539892286062, Validation accuracy 0.4772727272727273, lr [0.0001]
2024-06-02 14:47:49.600471 Epoch 10, Training loss 0.002401385921984911, Validation accuracy 0.5833333333333334, lr [1e-05]
2024-06-02 14:57:47.607993 Epoch 15, Training loss 0.0029041848611086607, Validation accuracy 0.6022727272727273, lr [1e-05]
Early stop on epoch 19
Weights are loaded from epoch 14


# Тестирование моделей

In [15]:
def test_model(model, dataloader: DataLoader):
    counter = 0
    for item, label in dataloader:
        item = item.to(device=device)
        label = label.to(device=device)
        print(model(item).argmax(1), label.argmax(1))
        if torch.equal(model(item).argmax(1), label.argmax(1)):
        #if model(item) == label:
            counter += 1
    return counter / len(dataloader.dataset)

In [16]:
test_model(value_resnet, train_dataloader)

tensor([ 2,  2, 12,  8,  0,  3, 10,  8,  1,  3, 12,  3,  3,  2, 12, 12],
       device='cuda:0') tensor([ 2,  2, 12,  1,  1, 11, 10,  8,  1,  3,  2,  3,  5, 11, 12, 12],
       device='cuda:0')
tensor([ 2,  2,  3,  6, 12,  0,  5,  0, 11,  9,  0,  9, 13,  1,  1,  6],
       device='cuda:0') tensor([ 2,  2,  3,  9,  7,  9,  5,  0,  1,  2,  0,  9, 13,  2,  1,  6],
       device='cuda:0')
tensor([ 2,  9,  5, 11,  3,  3,  3,  0,  2,  3,  1,  6, 13,  7,  9, 13],
       device='cuda:0') tensor([ 7,  8,  5, 11,  3,  7,  5, 10,  2,  3,  2,  6,  2,  7,  9, 13],
       device='cuda:0')
tensor([ 2,  1, 13, 13,  9, 10,  3,  2,  0,  7,  2, 12,  1,  6,  7,  4],
       device='cuda:0') tensor([ 2,  9,  2, 13,  9, 10,  3,  2,  9,  6, 12, 12,  1,  6,  5,  7],
       device='cuda:0')
tensor([10, 11,  2,  6,  3, 11,  6, 10,  5,  9,  3, 10,  3, 11,  9,  8],
       device='cuda:0') tensor([ 7,  6, 12,  6,  1,  8,  6, 10,  5,  6,  3, 10, 10, 11,  9, 10],
       device='cuda:0')
tensor([ 3,  2,  5,  3,  2,  6

0.0